In [16]:
import geopandas as gp
import pathlib

In [17]:
raw_full_gdf = gp.read_file("data/freemove_dlr_data/raw_full.geojson")
raw_points_gdf = gp.read_file("data/freemove_dlr_data/od_points.geojson")

raw_points_gdf

,uid,tid,leg_id,datetime,lng,lat,geometry
0,16643,987965,5762414,2022-11-01 14:04:58,13.286952,52.442537,POINT (13.28695 52.44254)
1,16643,987965,5762414,2022-11-01 14:11:35,13.280938,52.450522,POINT (13.28094 52.45052)
2,16643,987965,5762421,2022-11-01 14:32:15,13.331328,52.496161,POINT (13.33133 52.49616)
3,16643,987965,5762421,2022-11-01 14:56:44,13.347070,52.547154,POINT (13.34707 52.54715)
4,16643,995084,5789334,2022-11-04 13:47:02,13.409835,52.564417,POINT (13.40983 52.56442)
...,...,...,...,...,...,...,...
9911,17263,1002942,5817823,2022-11-07 09:59:42,13.468987,52.503399,POINT (13.46899 52.50340)
9912,17263,1002942,5817829,2022-11-07 10:23:36,13.541234,52.434862,POINT (13.54123 52.43486)
9913,17263,1002942,5817829,2022-11-07 10:31:36,13.536408,52.431660,POINT (13.53641 52.43166)
9914,17269,1003131,5818507,2022-11-07 15:01:30,13.326065,52.510312,POINT (13.32606 52.51031)


In [18]:
raw_points_gdf.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.